Start by reading the table

In [0]:
df = spark.read.table("bikes.bronze.crm_cust_info")

Import the _libraries_

In [0]:
from pyspark.sql.functions import col, trim, when
from pyspark.sql.types import StringType

triming the columns

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

Completeness for shortcut

In [0]:
# clean fun
df_clean = (
    df
    .withColumn("cst_key", trim(col("cst_key")))
    .withColumn("cst_firstname", trim(col("cst_firstname")))
    .withColumn("cst_lastname", trim(col("cst_lastname")))
    # Gender mapping
    .withColumn(
        "cst_gndr",
        when(col("cst_gndr") == "M", "Male")
        .when(col("cst_gndr") == "F", "Female")
        .otherwise(col("cst_gndr"))
    )
    # Marital status mapping
    .withColumn(
        "cst_marital_status",
        when(col("cst_marital_status") == "S", "Single")
        .when(col("cst_marital_status") == "M", "Married")
        .otherwise(col("cst_marital_status"))
    )
)
df = df_clean

Changing the columns names

In [0]:
rename_map = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}

df_renamed = df
for old, new in rename_map.items():
    df_renamed = df_renamed.withColumnRenamed(old, new)
df = df_renamed

Save the table and changing the table name

In [0]:
df.write.mode("overwrite").saveAsTable("bikes.silver.crm_customer_info")